In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cleaned-tmdb-movies/cleaned_tmdb_movies.csv


## Model Building

In [7]:
import numpy as np
import pandas as pd
new_df = pd.read_csv("/kaggle/input/cleaned-tmdb-movies/cleaned_tmdb_movies.csv")
def combine_features(row):
    return row['title']+' '+ row['tags']
new_df['combined_value'] = new_df.apply(combine_features, axis = 1)
new_df['index'] = [i for i in range(0, len(new_df))]

In [8]:
#Get title of movie
def title(index):
    return new_df[new_df.index == index]["title"].values[0]

#Get index of movie
def index(title):
    return new_df[new_df.title == title]["index"].values[0]

In [9]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
bert = SentenceTransformer('paraphrase-albert-small-v2')
#Get Embeddings
sentence_embeddings = bert.encode(new_df['combined_value'].tolist())
#Compute similarity
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(sentence_embeddings)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 4.5 MB/s eta 0:00:00a 0:00:01


/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/245 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/151 [00:00<?, ?it/s]

In [10]:
movie_name = 'Batman Begins'
movie_recommendation = sorted(list(enumerate(similarity[index(movie_name)])), key = lambda x:x[1], reverse = True)
print(title(movie_recommendation[1][0]), title(movie_recommendation[2][0]), title(movie_recommendation[3][0]), title(movie_recommendation[4][0]),title(movie_recommendation[5][0]), sep = "\n")

The Dark Knight
Batman v Superman: Dawn of Justice
The Dark Knight Rises
Batman: The Dark Knight Returns, Part 2
Batman


In [12]:
import joblib
joblib.dump(similarity, '/kaggle/working/similarity_matrix.pkl')

['/kaggle/working/similarity_matrix.pkl']

In [13]:
similarity = joblib.load('/kaggle/working/similarity_matrix.pkl')